In [1]:
# pip install MetaTrader5

In [2]:
import MetaTrader5 as mt5
import pandas as pd
import time
import os

In [3]:
# display data on the MetaTrader 5 package
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.45


In [4]:
# establish MetaTrader 5 connection to a specified trading account
if not mt5.initialize(login=4000013671, server="Darwinex-Live",password="K6Kpnug0RE6U"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

# display data on connection status, server name and trading account
print(mt5.terminal_info())
# display data on MetaTrader 5 version
print(mt5.version())

TerminalInfo(community_account=False, community_connection=False, connected=True, dlls_allowed=False, trade_allowed=False, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=True, build=3677, maxbars=100000, codepage=1252, ping_last=35325, community_balance=0.0, retransmission=0.6053447512180719, company='Tradeslide Trading Tech Limited', name='Darwinex MetaTrader 5', language='Spanish', path='C:\\Program Files\\Darwinex MetaTrader 5', data_path='C:\\Users\\OY\\AppData\\Roaming\\MetaQuotes\\Terminal\\6C3C6A11D1C3791DD4DBF45421BF8028', commondata_path='C:\\Users\\OY\\AppData\\Roaming\\MetaQuotes\\Terminal\\Common')
(500, 3677, '11 Apr 2023')


In [5]:
# connect to the trade account specifying a password and a server
authorized=mt5.login(login=4000013671, server="Darwinex-Live",password="K6Kpnug0RE6U")
if authorized:
    account_info=mt5.account_info()
    if account_info!=None:
        # display trading account data 'as is'
        print(account_info)
        # display trading account data in the form of a dictionary
        print("Show account_info()._asdict():")
        account_info_dict = mt5.account_info()._asdict()
        for prop in account_info_dict:
            print("  {}={}".format(prop, account_info_dict[prop]))
        print()
else:
    print("failed to connect to trade account 25115284 with password=gqz0343lbdm, error code =",mt5.last_error())

AccountInfo(login=4000013671, trade_mode=2, leverage=200, limit_orders=400, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=68472.46, credit=0.0, profit=0.0, equity=68472.46, margin=0.0, margin_free=68472.46, margin_level=0.0, margin_so_call=100.0, margin_so_so=50.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='NinjaSpeed_MT5', server='Darwinex-Live', currency='USD', company='Darwinex')
Show account_info()._asdict():
  login=4000013671
  trade_mode=2
  leverage=200
  limit_orders=400
  margin_so_mode=0
  trade_allowed=True
  trade_expert=True
  margin_mode=2
  currency_digits=2
  fifo_close=False
  balance=68472.46
  credit=0.0
  profit=0.0
  equity=68472.46
  margin=0.0
  margin_free=68472.46
  margin_level=0.0
  margin_so_call=100.0
  margin_so_so=50.0
  margin_initial=0.0
  margin_maintenance=0.0
  assets=0.0
  liabilities=0.0
  commission_blocked=0.0
  name

In [6]:
# Crear DataFrame
data = pd.DataFrame(index=['date'], columns=['bid', 'ask', 'MA_8', 'Derivada_8'])
data

,bid,ask,MA_8,Derivada_8
date,NaN,NaN,NaN,NaN


In [7]:
ric = 'NDX'
position = None

while True:
    os.system('cls')

    # Intentar habilitar la visualización de {ric} en MarketWatch
    selected = mt5.symbol_select(ric, True)
    if not selected:
        print(f"Failed to select {ric}")
        mt5.shutdown()
        quit()

    # Obtener el último tick de {ric}
    lasttick = mt5.symbol_info_tick(ric)

    # Verificar si el mercado está cerrado
    if lasttick.time_msc == 0:
        print("Mercado cerrado")
        time.sleep(60)
        continue

    # Crear un nuevo DataFrame con el registro actual
    nuevo_registro = pd.DataFrame({'date': [lasttick.time_msc],
                                   'bid': [lasttick.bid],
                                   'ask': [lasttick.ask],
                                   'MA_8': data['bid'].rolling(window=8).mean().iloc[-1] if len(data) > 0 else None,
                                   'Derivada_8': data['bid'].diff().rolling(window=8).mean().iloc[-1] if len(data) > 0 else None})

    # Establecer la columna 'date' como el índice
    nuevo_registro = nuevo_registro.set_index('date')

    # Concatenar el nuevo registro al DataFrame existente
    data = pd.concat([data, nuevo_registro])

    # Obtener el saldo disponible en la cuenta
    account_info = mt5.account_info()
    balance = account_info.balance
    equity = account_info.equity
    margin = account_info.margin

    # Calcular el tamaño de posición solo si el margen es distinto de cero
    if margin != 0:
        leverage = account_info.margin_mode
        margin_level = (equity / margin) * 100  # Nivel de margen en porcentaje
        max_position_size = (balance * leverage) / margin_level  # Tamaño máximo de posición
    else:
        max_position_size = 0  # O cualquier otro valor que desees asignar si el margen es cero

    # Imprimir resultados
    print("Saldo disponible:", balance)
    print("Equidad:", equity)
    print("Margen utilizado:", margin)
    print("Tamaño máximo de posición:", max_position_size)

    # Verificar la condición para enviar una orden de compra
    if data['Derivada_8'].iloc[-1] > 0 and position != 'BUY':
        if position == 'SELL':
            # Cerrar posición corta
            mt5.order_send(ric, 1, 0, 0, 0)  # Código 1 para cerrar la posición
        # Abrir posición larga
        mt5.order_send(ric, mt5.ORDER_TYPE_BUY, max_position_size, lasttick.ask, 0)
        position = 'BUY'

    # Verificar la condición para enviar una orden de venta
    if data['Derivada_8'].iloc[-1] < 0 and position != 'SELL':
        if position == 'BUY':
            # Cerrar posición larga
            mt5.order_send(ric, 1, 0, 0, 0)  # Código 1 para cerrar la posición
        # Abrir posición corta
        mt5.order_send(ric, mt5.ORDER_TYPE_SELL, max_position_size, lasttick.bid, 0)
        position = 'SELL'

    print(data.iloc[-1])

    time.sleep()

Saldo disponible: 68472.46
Equidad: 68472.46
Margen utilizado: 0.0
Tamaño máximo de posición: 0
bid           14466.5
ask           14467.3
MA_8              NaN
Derivada_8        NaN
Name: 1685670725691, dtype: float64


TypeError: time.sleep() takes exactly one argument (0 given)

In [ ]:
# shut down connection to the MetaTrader 5 terminal
mt5.shutdown()

True